In [1]:
%pip install langgraph langchain openai dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip show langgraph langchain openai dotenv requests

Name: langgraph
Version: 0.5.1
Summary: Building stateful, multi-actor applications with LLMs
Home-page: 
Author: 
Author-email: 
License-Expression: MIT
Location: c:\Users\aryan.dhull\OneDrive - Accenture\Desktop\LangGraph\env\Lib\site-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk, pydantic, xxhash
Required-by: 
---
Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\Users\aryan.dhull\OneDrive - Accenture\Desktop\LangGraph\env\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 
---
Name: openai
Version: 1.93.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: c:\Users\aryan.dhull\OneDrive - Accenture\Desktop\LangGraph\env

MODEL

In [4]:
from model import GroqChatModel
model = GroqChatModel()

In [ ]:
# Test model setup
response = model.invoke("Say hello to the world in a friendly way.")
print("✅ Model response:")
print(response.content)


✅ Model response:
Hello there! *big smile* It's so wonderful to meet you! I hope you're having an amazing day, and that life is treating you with kindness and joy. I'm thrilled to be a part of your day, even if it's just for a quick hello! Can't wait to chat with you and see what adventures we can have together!


STATE

In [6]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator

# Define the state for the travel planner application
class TravelPlannerState(TypedDict):
    destination: str
    budget: int
    flights: list[str]
    hotels: list[str]
    activities: list[str]
    include_hotels: bool
    messages: Annotated[list, add_messages]
    step_count: Annotated[int, operator.add]

NODES AND EDGES

In [7]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image

In [8]:
def add_destination_node(state: TravelPlannerState) -> TravelPlannerState: # type: ignore
    """Node to set travel destination and budget."""
    print("🌍 Welcome to the Travel Planner!")

    destination = input("Enter your travel destination: ")
    budget = int(input("Enter your budget for the trip: "))
    hotel_needed = input("Do you need hotels? (yes/no): ").strip().lower() == "yes"

    print(f"🎯 Setting destination to: {destination}")
    print(f"💰 Setting budget to: {budget}")
    if hotel_needed:
        include_hotels = True
        print("Hotels will be included in the search.")
    else:
        include_hotels = False
        print("Hotels will not be included in the search.")
    

    return {
        "destination": destination,
        "budget": budget,
        "include_hotels": include_hotels,
        "messages": [f"Destination set to {destination} with a budget of {budget}."],
        "step_count": 1
    }

In [9]:
def search_flights_node(state: TravelPlannerState) -> TravelPlannerState:
    """Node to search for flights based on destination"""
    destination = state.get("destination", "Unknown")
    response = model.invoke(f"List exactly 3 flight options to {destination} with prices. Format each as: 'Airline to {destination} - $price'. Only return the 3 lines, nothing else.")
    flights = [line.strip() for line in response.content.strip().split('\n') if line.strip() and '$' in line]
    
    print(f"✈️ Found {len(flights)} flights to {destination}")
    
    return {
        "flights": flights,
        "messages": [f"Found {len(flights)} flights to {destination}"],
        "step_count":1
    }

In [10]:
def search_hotels_node(state: TravelPlannerState) -> TravelPlannerState:
    """Node to search for hotels"""
    destination = state.get("destination", "Unknown")
    response = model.invoke(f"List exactly 3 hotel options in {destination} with prices per night. Format each as: 'Hotel Name in {destination} - $price/night'. Only return the 3 lines, nothing else.")
    hotels = [line.strip() for line in response.content.strip().split('\n') if line.strip() and '$' in line]
    
    print(f"🏨 Found {len(hotels)} hotels in {destination}")
    
    return {
        "hotels": hotels,
        "messages": [f"Found {len(hotels)} hotels in {destination}"],
        "step_count": 1
    }

In [11]:
def plan_activities_node(state: TravelPlannerState) -> TravelPlannerState:
    """Node to plan activities"""
    destination = state.get("destination", "Unknown")
    response = model.invoke(f"List exactly 3 tourist activities in {destination}. One activity per line. Only return the 3 activity names, nothing else.")
    activities = [line.strip() for line in response.content.strip().split('\n') if line.strip() and len(line.strip()) > 5]
    
    print(f"🎭 Planned {len(activities)} activities for {destination}")
    
    return {
        "activities": activities,
        "messages": [f"Planned {len(activities)} activities"],
        "step_count": 1
    }

In [12]:
def branch_after_destination(state):
        # If want_hotels is True, go to both flights and hotels in parallel
        # If want_hotels is False, only go to search_flights
        if state.get("include_hotels", True):
            return "flights_and_hotels"
        else:
            return "flights_only"

In [13]:
def flights_and_hotels_router(state):
    return state

In [14]:
def summarize_plan_node(state: TravelPlannerState) -> TravelPlannerState: # type: ignore
    """Final node to summarize the travel plan"""
    destination = state.get("destination", "Unknown")

    total_steps = state.get("step_count", 0)
    
    summary = f"""
    📋 TRAVEL PLAN SUMMARY for {destination}:
    ✈️ Flights: {len(state.get('flights', []))} options
    🏨 Hotels: {len(state.get('hotels', []))} options  
    🎭 Activities: {len(state.get('activities', []))} planned
    📊 Total processing steps: {total_steps}
    """
    
    print(summary)

    return {
        "messages": [summary],
        "step_count": 1
    }

In [15]:
def create_travel_planner_graph():
    """Create and configure the LangGraph workflow"""
    
    # Initialize the graph with our state schema
    workflow = StateGraph(TravelPlannerState) # type: ignore
    workflow.name = "Travel Planner Workflow"

    # Defining the nodes in the workflow
    workflow.add_node("add_destination", add_destination_node)
    workflow.add_node("flights_and_hotels_router", flights_and_hotels_router)
    workflow.add_node("plan_activities", plan_activities_node)
    workflow.add_node("search_flights", search_flights_node)
    workflow.add_node("search_hotels", search_hotels_node)
    workflow.add_node("summarize", summarize_plan_node)
    

    # Adding edges to connect the nodes
    workflow.add_edge(START, "add_destination")

    # #Parallel execution of flight and hotel searches
    # workflow.add_edge("add_destination", "search_flights")
    # workflow.add_edge("add_destination", "search_hotels")
    
    # Conditional branching based on whether hotels are included
    workflow.add_conditional_edges(
        "add_destination",
        branch_after_destination,
        {
            "flights_and_hotels": "flights_and_hotels_router",
            "flights_only": "search_flights"
        }
    )

    workflow.add_edge("flights_and_hotels_router", "search_flights")
    workflow.add_edge("flights_and_hotels_router", "search_hotels")
    workflow.add_edge("flights_and_hotels_router", "plan_activities")  

    # If only flights, plan_activities must still run
    workflow.add_edge("search_flights", "plan_activities")

    # All paths converge to summarize
    workflow.add_edge("search_flights", "summarize")
    workflow.add_edge("search_hotels", "summarize")
    workflow.add_edge("plan_activities", "summarize")

    workflow.add_edge("summarize", END)

    return workflow

In [16]:
if __name__ == "__main__":
    print("🚀 Creating LangGraph Travel Planner...")
    
    # Create the graph
    travel_app = create_travel_planner_graph().compile()
    
    # Initial state
    initial_state = {
        "destination": "",
        "budget": 0,
        "flights": [],
        "hotels": [],
        "activities": [],
        "include_hotels": True,  # Default to including hotels
        "messages": [],
        "step_count": 0
    }
    
    print("\n📊 INITIAL STATE STRUCTURE:")
    for key, value in initial_state.items():
        print(f"  {key}: {type(value).__name__} = {value}")
    
    print("\n" + "="*50)
    print("🔄 EXECUTING LANGGRAPH WORKFLOW...")
    print("="*50)
    
    # Run the workflow
    final_state = travel_app.invoke(initial_state)
    
    print("\n" + "="*50)
    print("📊 FINAL STATE STRUCTURE:")
    print("="*50)
    for key, value in final_state.items():
        if key == "messages":
            print(f"  {key}: {len(value)} messages")
            for i, msg in enumerate(value, 1):
                print(f"    {i}. {msg}")
        elif isinstance(value, list):
            print(f"  {key}: {len(value)} items")
            for item in value:
                print(f"    - {item}")
        else:
            print(f"  {key}: {value}")
    
    print(f"\n✅ Workflow completed with {final_state['step_count']} total steps")

🚀 Creating LangGraph Travel Planner...

📊 INITIAL STATE STRUCTURE:
  destination: str = 
  budget: int = 0
  flights: list = []
  hotels: list = []
  activities: list = []
  include_hotels: bool = True
  messages: list = []
  step_count: int = 0

🔄 EXECUTING LANGGRAPH WORKFLOW...
🌍 Welcome to the Travel Planner!
🎯 Setting destination to: Vietnam
💰 Setting budget to: 3000
Hotels will be included in the search.
🎭 Planned 4 activities for Vietnam
✈️ Found 3 flights to Vietnam
🏨 Found 3 hotels in Vietnam

    📋 TRAVEL PLAN SUMMARY for Vietnam:
    ✈️ Flights: 3 options
    🏨 Hotels: 3 options  
    🎭 Activities: 4 planned
    📊 Total processing steps: 5
    
🎭 Planned 4 activities for Vietnam

    📋 TRAVEL PLAN SUMMARY for Vietnam:
    ✈️ Flights: 3 options
    🏨 Hotels: 3 options  
    🎭 Activities: 4 planned
    📊 Total processing steps: 7
    

📊 FINAL STATE STRUCTURE:
  destination: Vietnam
  budget: 3000
  flights: 3 items
    - Qantas to Vietnam - $550
    - Thai Airways to Vietnam -